In [4]:
import pandas as pd
import numpy as np
import scorecardpy as sc
from timeit import default_timer as timer

In [8]:
def cal_woebin_iv(file_path,y,no_of_col_to_read_at_time=2,min_iv=0.2):
    """
    calculate the WoE and IV for all features of given Dataframe.It uses scorecardpy library.

    Parameters
    ----------
    file_path:str
        Input file path
    y:str
        Dependent variable name.
    no_of_col_to_read_at_time : int
        It specify the number of columns to read in instead of reading whole dataset.
    min_iv : float
        minimum value of IV to select the feature
    """
    start = timer()
    
    iv_new = []
    for group in chunker(cols, no_of_col_to_read_at_time):
        group.append(y) 
        print(group)
        df = pd.read_parquet(file_path,columns=group)
        print(df.shape)
        # woe binning ------
        bins = sc.woebin(df, y=y)
        iv_new.append(bins)

    flatten_iv_new ={k: v for d in iv_new for k, v in d.items()}
    col_selected = []
    for key in flatten_iv_new:
        iv_val = flatten_iv_new[key]['total_iv'][0]
        #print(iv_val)
        if ((iv_val >= min_iv) & (iv_val <= 0.5)):
            col_selected.append(key)
    end = timer()
    print("Time to run the function is : ",end-start)
    return col_selected,flatten_iv_new

def chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))






In [7]:
def woe_encoding(df,bins_dict):
    # converting to bin
    dt_bin = sc.woebin_ply(df, bins=bins_dict, to = 'bin')
    str(dt_bin)
    return dt_bin


In [27]:
input_file_path_parquet = 'loan-test.parquet'
input_file_path_csv = 'loan-test.csv'
dependent_var = 'Credit_History'
cols = pd.read_csv(input_file_path_csv, index_col=0, nrows=0).columns.tolist()
cols.remove(dependent_var)
selected_features,bins = cal_woebin_iv(file_path= input_file_path_parquet,y=dependent_var,no_of_col_to_read_at_time=4,min_iv=0.1)
print(len(selected_features))
print(selected_features)
df = pd.read_parquet(input_file_path_parquet,columns=selected_features)
woe_encoded_df = woe_encoding(df.copy(),bins_dict=bins)

['Gender', 'Married', 'Dependents', 'Education', 'Credit_History']
(367, 5)
[INFO] creating woe binning ...


/home/amol-murme/SBM/myenv/lib/python3.10/site-packages/scorecardpy/condition_fun.py:110: UserWarning: There are NaNs in 'Credit_History' column. The rows with NaN in 'Credit_History' were removed from dat.
  warnings.warn("There are NaNs in \'{}\' column. The rows with NaN in \'{}\' were removed from dat.".format(y,y))
/home/amol-murme/SBM/myenv/lib/python3.10/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


['Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Credit_History']
(367, 5)
[INFO] creating woe binning ...
['Loan_Amount_Term', 'Property_Area', 'Credit_History']
(367, 3)
[INFO] creating woe binning ...
Time to run the function is :  3.310724788999323
4
['Self_Employed', 'CoapplicantIncome', 'ApplicantIncome', 'LoanAmount']
[INFO] converting into woe values ...


In [28]:
woe_encoded_df

,Self_Employed_woe,CoapplicantIncome_woe,ApplicantIncome_woe,LoanAmount_woe
0,0.026163,0.070470,-0.087337,0.126859
1,0.026163,0.329057,0.267938,-0.195551
2,0.026163,-0.167380,-0.087337,-0.519601
3,0.026163,-0.637384,-0.814007,0.126859
4,0.026163,0.070470,0.267938,0.844221
...,...,...,...,...
362,1.249686,0.329057,0.267938,0.126859
363,0.026163,0.070470,0.267938,0.126859
364,0.026163,-0.167380,0.267938,-0.195551
365,0.026163,-0.637384,-0.087337,-0.519601


In [7]:
woe_encoded_df['M273_woe'].value_counts()

-0.155824    1855280
 0.896262     134315
 0.086003     123780
 0.439802     112277
Name: M273_woe, dtype: int64

In [9]:
bins['M273']

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,M273,"[-inf,-0.5499999999999996)",112277,0.050447,112153,124,0.001104,0.439802,0.012251,0.109795,-0.5499999999999996,False
1,M273,"[-0.5499999999999996,0.4500000000000013)",1855280,0.833589,1854150,1130,0.000609,-0.155824,0.018744,0.109795,0.4500000000000013,False
2,M273,"[0.4500000000000013,0.7000000000000015)",123780,0.055615,123684,96,0.000776,0.086003,0.000430,0.109795,0.7000000000000015,False
3,M273,"[0.7000000000000015,inf)",134315,0.060349,134081,234,0.001742,0.896262,0.078370,0.109795,inf,False


In [10]:
del woe_encoded_df

In [6]:
def one_hot_encoding(df,bins):
    df_transform = pd.DataFrame()
    categorical_cols = df.select_dtypes(include=['object','category']).columns.tolist()  # include 'bool' and 'category' dtype
    print(categorical_cols)
    if categorical_cols:
        df_transform = pd.get_dummies(df[categorical_cols], columns = categorical_cols)
    cols = df.select_dtypes(exclude=['object','category']).columns.tolist()  # include 'bool' and 'category' dtype
    print(cols)
    for col in cols:
        temp = bins[col]
        for i in range(len(temp)):
            l= (bins[col]['bin'][i]).replace('[','').replace(')','').split(',')
            if l[0] != 'missing':
                min = float(l[0])
                max = float(l[1])
                df_transform[col + '_' +temp['bin'][i]] = np.where((df[col] <= max) & (df[col] > min),1,0)
                

    return df_transform


In [63]:
min = float('-inf')
max = 2600
df['ApplicantIncom_oneHotEncoded'] = np.where()

In [20]:
df = pd.read_parquet(input_file_path_parquet,columns=selected_features)

In [33]:
one_hot_encoded_df = one_hot_encoding(df,bins)

['Self_Employed']
['CoapplicantIncome', 'ApplicantIncome', 'LoanAmount']


In [34]:
one_hot_encoded_df

,Self_Employed_No,Self_Employed_Yes,"CoapplicantIncome_[-inf,800.0)","CoapplicantIncome_[800.0,1800.0)","CoapplicantIncome_[1800.0,2200.0)","CoapplicantIncome_[2200.0,3000.0)","CoapplicantIncome_[3000.0,4200.0)","CoapplicantIncome_[4200.0,inf)","ApplicantIncome_[-inf,2600.0)","ApplicantIncome_[2600.0,4600.0)","ApplicantIncome_[4600.0,6600.0)","ApplicantIncome_[6600.0,inf)","LoanAmount_[-inf,80.0)","LoanAmount_[80.0,125.0)","LoanAmount_[125.0,155.0)","LoanAmount_[155.0,210.0)","LoanAmount_[210.0,inf)"
0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
2,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0
4,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
363,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
364,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
365,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0


In [28]:
# Defining a positive infinite integer
positive_infinity = float('inf')
print('Positive Infinity: ', positive_infinity)

# Defining a negative infinite integer
negative_infinity = float('-inf')
print('Negative Infinity: ', negative_infinity)


Positive Infinity:  inf
Negative Infinity:  -inf


In [4]:
t = []
if t:
    print('yes')
else:
    print('no')

no


In [45]:
l= (bins['ApplicantIncome']['bin'][0]).replace('[','').replace(')','').split(',')

In [47]:
min = float(l[0])
max = float(l[1])

### test on macro_input.csv

In [9]:
input_file_path_parquet = 'macro_input_new.parquet'
input_file_path_csv = 'macro_input_new.csv'
dependent_var = 'def_trig'
cols = pd.read_csv(input_file_path_csv, index_col=0, nrows=0).columns.tolist()
cols.remove(dependent_var)
selected_features,bins = cal_woebin_iv(file_path= input_file_path_parquet,y=dependent_var,no_of_col_to_read_at_time=6,min_iv=0.1)
print(len(selected_features))
print(selected_features)
df = pd.read_parquet(input_file_path_parquet,columns=selected_features)
woe_encoded_df = woe_encoding(df.copy(),bins_dict=bins)

['M98', 'M99', 'M100', 'M194', 'M104', 'M105', 'def_trig']
(2225652, 7)
[INFO] creating woe binning ...
Binning on 2225652 rows and 7 columns in 00:00:46
['M106', 'M107', 'M108', 'M109', 'M110', 'M111', 'def_trig']
(2225652, 7)
[INFO] creating woe binning ...
Binning on 2225652 rows and 7 columns in 00:00:42
['M205', 'M206', 'M207', 'M115', 'M209', 'M117', 'def_trig']
(2225652, 7)
[INFO] creating woe binning ...
Binning on 2225652 rows and 7 columns in 00:00:41
['M118', 'M119', 'M120', 'M121', 'M122', 'M216', 'def_trig']
(2225652, 7)
[INFO] creating woe binning ...
Binning on 2225652 rows and 7 columns in 00:00:41
['M218', 'M126', 'M127', 'M221', 'M222', 'M223', 'def_trig']
(2225652, 7)
[INFO] creating woe binning ...
Binning on 2225652 rows and 7 columns in 00:00:42
['M131', 'M132', 'M133', 'M134', 'M135', 'M229', 'def_trig']
(2225652, 7)
[INFO] creating woe binning ...
Binning on 2225652 rows and 7 columns in 00:00:41
['M230', 'M138', 'M139', 'M140', 'M141', 'M142', 'def_trig']
(2225

In [10]:
woe_encoded_df

,M334_woe,M291_woe,M314_woe,M346_woe,MOB_woe,M319_woe,M269_woe,M320_woe,M325_woe,M321_woe,M169_woe,Date.1_woe,M273_woe,M272_woe,M159_woe,M293_woe
0,-0.092233,-0.08536,0.038099,-0.124949,0.365087,-0.60252,-0.912033,-0.60252,-0.335096,-0.195895,-0.03266,0.166219,-0.155824,-0.228504,-0.044284,-0.233132
1,-0.092233,-0.08536,0.038099,-0.124949,0.365087,-0.60252,-0.912033,-0.60252,-0.335096,-0.195895,-0.03266,0.166219,-0.155824,-0.228504,-0.044284,-0.233132
2,-0.092233,-0.08536,0.038099,-0.124949,0.365087,-0.60252,-0.912033,-0.60252,-0.335096,-0.195895,-0.03266,0.166219,-0.155824,-0.228504,-0.044284,-0.233132
3,-0.092233,-0.08536,0.038099,-0.124949,0.365087,-0.60252,-0.912033,-0.60252,-0.335096,-0.195895,-0.03266,0.166219,-0.155824,-0.228504,-0.044284,-0.233132
4,-0.092233,-0.08536,0.038099,-0.124949,0.365087,-0.60252,-0.912033,-0.60252,-0.335096,-0.195895,-0.03266,0.166219,-0.155824,-0.228504,-0.044284,-0.233132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225647,-0.095722,-0.08536,-0.258150,-0.124949,0.365087,-0.07245,-0.314526,-0.07245,0.125528,-0.014213,-0.03266,0.115219,-0.155824,-0.124180,0.603371,-0.679286
2225648,-0.095722,-0.08536,-0.258150,-0.124949,0.365087,-0.07245,-0.314526,-0.07245,0.125528,-0.014213,-0.03266,0.115219,-0.155824,-0.124180,0.603371,-0.679286
2225649,-0.095722,-0.08536,-0.258150,-0.124949,0.365087,-0.07245,-0.314526,-0.07245,0.125528,-0.014213,-0.03266,0.115219,-0.155824,-0.124180,0.603371,-0.679286
2225650,-0.095722,-0.08536,-0.258150,-0.124949,0.022864,-0.07245,-0.314526,-0.07245,0.125528,-0.014213,-0.03266,0.115219,-0.155824,-0.124180,0.603371,-0.679286


In [11]:
one_hot_encoded_df = one_hot_encoding(df,bins)

[]
['M159', 'M169', 'M269', 'M272', 'M273', 'Date.1', 'M291', 'M293', 'M314', 'M319', 'M320', 'M325', 'M321', 'M334', 'M346', 'MOB']


In [12]:
one_hot_encoded_df

,"M159_[-inf,-0.1999999999999984)","M159_[-0.1999999999999984,-0.09999999999999831)","M159_[-0.09999999999999831,1.7763568394002505e-15)","M159_[1.7763568394002505e-15,0.30000000000000204)","M159_[0.30000000000000204,0.40000000000000213)","M159_[0.40000000000000213,1.0000000000000027)","M159_[1.0000000000000027,1.8000000000000034)","M159_[1.8000000000000034,inf)","M169_[-inf,-8.0)","M169_[-8.0,-5.0)",...,"M346_[-inf,-1.5)","M346_[-1.5,0.0)","M346_[0.0,5.0)","M346_[5.0,9.5)","M346_[9.5,inf)","MOB_[-inf,5.0)","MOB_[5.0,10.0)","MOB_[10.0,30.0)","MOB_[30.0,70.0)","MOB_[70.0,inf)"
0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225647,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2225648,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2225649,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2225650,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
